<a href="https://colab.research.google.com/github/doctorpeppercokebear/DEEPLEANNING/blob/tensorflow/%5B%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8%5DCh01_%E1%84%8C%E1%85%A6%E1%84%8C%E1%85%A9%E1%84%80%E1%85%A9%E1%86%BC%E1%84%8C%E1%85%A5%E1%86%BC_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%8B%E1%85%A3_10_%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5_%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%85%E1%85%B5%E1%84%92%E1%85%A1%E1%84%80%E1%85%B5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFRecord Builder

In [ ]:
import glob
import os
import tensorflow as tf
import cv2

## Paths and Hyperparameters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ./drive/MyDrive/DL/dataset2.7z ./

In [ ]:
!apt-get install p7zip-full


Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-7build1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
!7zr x ./dataset2.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 593632225 bytes (567 MiB)

Extracting archive: ./dataset2.7z
--
Path = ./dataset2.7z
Type = 7z
Physical Size = 593632225
Headers Size = 39347
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      5% 94 - dataset/2/FAIL/0011.png                                   5% 154 - dataset/2/FAIL/0071.png                                    6% 185 - dataset/2/FAIL/0102.png                                    7% 264 - dataset/2/OK/0062.png                                

In [ ]:
DATASET_OK_PATTERN = 'dataset/3/OK/*.png'
DATASET_FAIL_PATTERN = 'dataset/3/FAIL/*.png'

TFRECORD_PATH = 'dataset/tfrecords/'
IMAGE_PER_TFRECORD = 100

## Import data

In [ ]:
ok_list = glob.glob(DATASET_OK_PATTERN)
fail_list = glob.glob(DATASET_FAIL_PATTERN)
num_ok = len(ok_list)
num_fail = len(fail_list)
print(num_ok, num_fail)

1240 359


In [ ]:
# Oversampling
fail_list_new = list()
for _ in range(num_ok // num_fail):
  fail_list_new += fail_list
fail_list_new += fail_list[:num_ok % num_fail]
fail_list = fail_list_new
print(len(ok_list), len(fail_list))

1240 1240


In [ ]:
ok_label = [0] * len(ok_list)
fail_label = [1] * len(fail_list)

In [ ]:
file_list = ok_list + fail_list
label_list = ok_label + fail_label

In [ ]:
print(len(file_list), len(label_list))

2480 2480


## TFRecord functions

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

## Write TFRecords

In [ ]:
if os.path.exists(TFRECORD_PATH) is False:
  os.mkdir(TFRECORD_PATH)

num_tfrecords = len(file_list) // IMAGE_PER_TFRECORD
if len(file_list) % IMAGE_PER_TFRECORD != 0:
  num_tfrecords += 1
print(num_tfrecords)

25


In [ ]:
for idx in range(num_tfrecords):
  idx0 = idx * IMAGE_PER_TFRECORD
  idx1 = idx0 + IMAGE_PER_TFRECORD
  record_file = TFRECORD_PATH + '%5d.tfrecords' % idx
  # print(idx, idx0, idx1, record_file)
  with tf.io.TFRecordWriter(record_file) as writer:
    for filename, label in zip(file_list[idx0:idx1],
                               label_list[idx0:idx1]):
      image_string = open(filename, 'rb').read()
      tf_example = image_example(image_string, label)
      writer.write(tf_example.SerializeToString())

In [ ]:
!7zr a dataset3.7z ./dataset/MyDrive/


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Open archive: dataset3.7z
--
Path = dataset3.7z
Type = 7z
Physical Size = 788554188
Headers Size = 39738
Method = LZMA2:24
Solid = +
Blocks = 1

Scanning the drive:
  0M Scan  ./dataset/                     
./dataset/MyDrive

0 files, 0 bytes

Updating archive: dataset3.7z

Items to compress: 0

  0%     20% = dataset/tfrecords/   24.tfrecords                                         32% = dataset/tfrecords/   24.tfrecords                                         42% = dataset/tfrecords/   24.tfrecords                                        

In [ ]:
!cp ./dataset3.7z ./drive/MyDrive/